In [1]:
import os

In [8]:
data_dir = "/home/lacie/Github/MasterStudy/ComputerVision/data/Simplycity/"

In [9]:
impath= os.listdir(data_dir)

In [10]:
impath

['6', '0', '7', '1', '4', '5', '3', '2', '9', '8']

In [12]:
!pip install opencv-python
!pip install opencv-contrib-python

  Using cached opencv_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.5 MB)
     |████████████████████████████████| 66.7 MB 2.4 MB/s eta 0:00:01     |██████▋                         | 13.8 MB 4.7 MB/s eta 0:00:12
